In [1]:
import transformers

In [37]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration

# Load the tokenizer and model
tokenizer = T5TokenizerFast.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [38]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

In [39]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 60,801,536 || trainable%: 0.4850


In [77]:
from datasets import load_dataset
dataset = load_dataset("cnn_dailymail","3.0.0")

#stan_dataset = load_dataset("squad")
#quora_dataset = load_dataset("quora")

ValueError: Config name is missing.
Please pick one among the available configs: ['1.0.0', '2.0.0', '3.0.0']
Example of usage:
	`load_dataset('cnn_dailymail', '1.0.0')`

In [52]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [71]:
def tokenization(example):
    inputs = tokenizer(example["article"], padding="max_length", truncation="only_first", max_length=512)
    outputs = tokenizer(example["highlights"], padding="max_length", truncation="only_first", max_length=128)

    
    inputs["labels"] = outputs["input_ids"]
    
    return inputs
    
#len(raw_datasets["train"]), len(train_dataset)

dataset_train = dataset['train'].select(range(1000)).map(tokenization, batched=True)
dataset_val = dataset['validation'].select(range(1000)).map(tokenization, batched=True)
dataset_test = dataset['test'].select(range(1000)).map(tokenization, batched=True)

#trans_dataset_train = trans_data['train'].map(tokenization, batched=True)
#trans_dataset_val = trans_data['validation'].map(tokenization, batched=True)
#trans_dataset_test = trans_data['test'].map(tokenization, batched=True)

In [72]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [73]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [76]:
training_args = transformers.TrainingArguments(
    output_dir="your-name/bigscience/mt0-large-lora",
    learning_rate=1e-3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipykernel_4171779/499583567.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/home/agrawal/anaconda3/envs/kan/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.92 GiB. GPU 0 has a total capacity of 39.39 GiB of which 1.48 GiB is free. Process 3167150 has 7.55 GiB memory in use. Process 3273128 has 2.50 GiB memory in use. Including non-PyTorch memory, this process has 27.86 GiB memory in use. Of the allocated memory 20.72 GiB is allocated by PyTorch, and 4.81 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)